In [3]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error as mse

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
### READ DATA ###
cols = ["Zeitraum", "T_Solar_VL", "T_Solar_RL", "Vd_Solar", "T_FBH_VL_env","T_FBH_RL", "T_FBH_RL_env", "Vd_FBH_env", "T_Nachheizung_VL", "T_Nachheizung_RL", "Vd_Nachheizung", "T_WW_VL", "T_WW_RL", "Vd_WW", "T_P_top" ,"T_P_o", "T_P_mo", "T_P_u", "T_P_mu"]
df = pd.read_csv('../../../Data/P2017_20_Solarhouse_2.csv', usecols=cols, sep=';')
df['Zeitraum'] = pd.to_datetime(df['Zeitraum'])
df.drop_duplicates('Zeitraum', inplace=True)
df.set_index('Zeitraum', inplace=True)

print(df.shape)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'P2017_20_Solarhouse_2.csv'

In [ ]:
### INSERT MISSING DATES ###

df = df.reindex(pd.date_range(df.head(1).index[0], df.tail(1).index[0], freq='H'))

df.shape

In [ ]:
### PLOT TRAFFIC SAMPLE ###
# df_part = .T_FBH_VL
# print(df_part)
df.T_FBH_RL_env.plot(figsize=(18,5))
plt.ylabel('T_FBH_RL_env')
df.T_FBH_RL.plot(figsize=(18,5))
plt.ylabel('T_FBH_RL')

In [ ]:
### PLOT MISSING VALUES OVER TIME ###

plt.figure(figsize=(18,5))
sns.heatmap(df[['T_FBH_RL_env']].isna().T, cbar=False, cmap='plasma', 
            xticklabels=False, yticklabels=['T_FBH_RL_env'])
plt.xticks(range(0,len(df), 24*180), list(df.index.year[::24*180]))
np.set_printoptions(False)

In [ ]:
### FILL MISSING VALUES ###

df = df[df.index.year.isin([2019])].copy()

df = pd.concat([df.select_dtypes(include=['object']).fillna(method='backfill'),
                df.select_dtypes(include=['float']).interpolate()], axis=1)

df.shape

In [ ]:
### PLOT TRAFFIC DISTRIBUTION IN EACH MONTH ###

plt.figure(figsize=(9,5))
sns.boxplot(x=df.index.month, y=df.T_FBH_RL_env, palette='plasma')

plt.ylabel('T_FBH_RL_env'); plt.xlabel('month')

In [ ]:
### PLOT TRAFFIC DISTRIBUTION IN EACH WEEKDAY ###

plt.figure(figsize=(9,5))
sns.boxplot(x=df.index.weekday, y=df.T_FBH_RL_env, palette='plasma')

plt.ylabel('T_FBH_RL_env'); plt.xlabel('weekday')

In [ ]:
### PLOT TRAFFIC DISTRIBUTION IN EACH HOUR ###

plt.figure(figsize=(9,5))
sns.boxplot(x=df.index.hour, y=df.T_FBH_RL_env, palette='plasma')

plt.ylabel('T_FBH_RL_env'); plt.xlabel('hour')